In [0]:
from pyspark.sql.functions import monotonically_increasing_id, col

# 1. Leitura dos dados carregados manualmente (Origem)
# O Databricks reconhece o upload manual como uma tabela registrada.
try:
    df_bronze = spark.read.table("workspace.default.student_mat")
except:
    # Alternativa de segurança
    df_bronze = spark.read.table("default.student_mat")

print("Dados carregados com sucesso. Visualizando esquema original:")
df_bronze.printSchema()

# 2. Tratamento: Adicionar ID único e Renomear Colunas
# Como o dataset original não tem ID, criamos um para permitir o relacionamento futuro
df_silver = df_bronze.withColumn("id_aluno", monotonically_increasing_id()) \
    .withColumnRenamed("school", "escola") \
    .withColumnRenamed("sex", "sexo") \
    .withColumnRenamed("age", "idade") \
    .withColumnRenamed("address", "tipo_endereco") \
    .withColumnRenamed("famsize", "tamanho_familia") \
    .withColumnRenamed("Pstatus", "status_coabitacao_pais") \
    .withColumnRenamed("Medu", "educacao_mae") \
    .withColumnRenamed("Fedu", "educacao_pai") \
    .withColumnRenamed("Mjob", "trabalho_mae") \
    .withColumnRenamed("Fjob", "trabalho_pai") \
    .withColumnRenamed("reason", "razao_escolha_escola") \
    .withColumnRenamed("guardian", "guardiao") \
    .withColumnRenamed("traveltime", "tempo_viagem") \
    .withColumnRenamed("studytime", "tempo_estudo") \
    .withColumnRenamed("failures", "reprovacoes") \
    .withColumnRenamed("schoolsup", "suporte_escolar") \
    .withColumnRenamed("famsup", "suporte_familiar") \
    .withColumnRenamed("paid", "aulas_extras_pagas") \
    .withColumnRenamed("activities", "atividades_extracurriculares") \
    .withColumnRenamed("nursery", "frequentou_creche") \
    .withColumnRenamed("higher", "quer_curso_superior") \
    .withColumnRenamed("internet", "possui_internet") \
    .withColumnRenamed("romantic", "relacionamento_amoroso") \
    .withColumnRenamed("famrel", "qualidade_relacao_familiar") \
    .withColumnRenamed("freetime", "tempo_livre") \
    .withColumnRenamed("goout", "sair_com_amigos") \
    .withColumnRenamed("Dalc", "alcool_dia_util") \
    .withColumnRenamed("Walc", "alcool_fim_semana") \
    .withColumnRenamed("health", "saude") \
    .withColumnRenamed("absences", "faltas") \
    .withColumnRenamed("G1", "nota_g1") \
    .withColumnRenamed("G2", "nota_g2") \
    .withColumnRenamed("G3", "nota_final")

# 3. Removendo Duplicadas por linhas identicas
df_silver = df_silver.dropDuplicates()

# 4. Salvar como Tabela Gerenciada (Managed Table)
table_name = "default.silver_student_performance"
df_silver.write.format("delta").mode("overwrite").saveAsTable(table_name)

print(f"Tabela Silver salva com sucesso: {table_name}")
display(spark.sql(f"SELECT * FROM {table_name} LIMIT 5"))

Dados carregados com sucesso. Visualizando esquema original:
root
 |-- school: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- age: long (nullable = true)
 |-- address: string (nullable = true)
 |-- famsize: string (nullable = true)
 |-- Pstatus: string (nullable = true)
 |-- Medu: long (nullable = true)
 |-- Fedu: long (nullable = true)
 |-- Mjob: string (nullable = true)
 |-- Fjob: string (nullable = true)
 |-- reason: string (nullable = true)
 |-- guardian: string (nullable = true)
 |-- traveltime: long (nullable = true)
 |-- studytime: long (nullable = true)
 |-- failures: long (nullable = true)
 |-- schoolsup: string (nullable = true)
 |-- famsup: string (nullable = true)
 |-- paid: string (nullable = true)
 |-- activities: string (nullable = true)
 |-- nursery: string (nullable = true)
 |-- higher: string (nullable = true)
 |-- internet: string (nullable = true)
 |-- romantic: string (nullable = true)
 |-- famrel: long (nullable = true)
 |-- freetime: long (nulla

escola,sexo,idade,tipo_endereco,tamanho_familia,status_coabitacao_pais,educacao_mae,educacao_pai,trabalho_mae,trabalho_pai,razao_escolha_escola,guardiao,tempo_viagem,tempo_estudo,reprovacoes,suporte_escolar,suporte_familiar,aulas_extras_pagas,atividades_extracurriculares,frequentou_creche,quer_curso_superior,possui_internet,relacionamento_amoroso,qualidade_relacao_familiar,tempo_livre,sair_com_amigos,alcool_dia_util,alcool_fim_semana,saude,faltas,nota_g1,nota_g2,nota_final,id_aluno
GP,F,18,R,GT3,T,4,4,teacher,other,reputation,mother,2,2,0,no,no,yes,yes,yes,yes,yes,no,4,3,4,2,2,4,8,12,10,11,267
GP,F,17,U,LE3,T,4,2,teacher,services,reputation,mother,1,4,0,no,yes,yes,yes,yes,yes,yes,no,4,2,3,1,1,4,6,14,12,13,256
GP,F,18,U,GT3,T,2,1,other,other,course,other,2,3,0,no,yes,yes,no,no,yes,yes,yes,4,4,4,1,1,3,0,7,0,0,244
GP,F,17,U,GT3,T,4,3,health,services,reputation,mother,1,3,0,no,yes,yes,no,yes,yes,yes,no,4,2,2,1,2,3,0,15,15,15,291
GP,M,17,U,GT3,T,3,3,other,other,reputation,father,1,2,0,no,no,no,yes,no,yes,yes,no,4,3,4,1,4,4,4,6,5,6,177
